# Linear Programming: Rock, Paper, Scissors

I am going to go through an old blog post of mine where I ran the Rock, Paper, Scissors game through a linear program. The original post was because I saw a *simple* linear program in the book but I needed a step hard to see how I was going to scale up the variables.

**CVXOPT**  
[CVXOPT](https://cvxopt.org/) is an open source package for convex optimizations in Python. At the time of my research this was by far the fastest library by an order of magnitude. But, being on Windows I didn't much speed up. In the context of this notebook the speed doesn't really matter.

**Installation Instructions**  
I was able to install this by running a pip install as listed [here](https://cvxopt.org/install/index.html).

In [1]:
from cvxopt import matrix, solvers
glpksolver = 'cvxopt_glpk'
solvers.options['show_progress'] = False
solvers.options['LPX_K_MSGLEV'] = 0

**LP Solver**  
cvxopt has a solver class that solves a linear equation by passing in c, G, and h. The program will  

minimize $$c^{T}x$$ subject to $$Gx + s = h$$ $$Ax = b$$ $$s>=0$$

**c Matrix**  
I need to *minimize* the results so I need to have a -1 in the first column that represents U while Rock, Paper, and Scissor columns are all 0.

In [3]:
# Minimize: U + R + P + S
c = matrix([-1.,0.,0.,0.])

**G Matrix**  
This is the big matrix. It contains ALL of the requirements on my linear program. 

The first 3 equations set up the results.  

The first row is throwing Rock where do nothing with Rock (0), you lose (-1) to paper, but win (+1) to scissors. The second row is for Paper. The third row is for Scissors.  

Throwing Rock: U - P + S <= 0 -> [1,0,-1,1]  
Throwing Paper: U + R - S <= 0 -> [1,1,0,-1]  
Throwing Scissors: U - R + P <= 0 -> [1,-1,1,0]  

The next 3 rows is to ensure you can only throw a single play. Now, since we are using cvxopt we need to negate these. Instead of R >= 0 we need -R <= 0  

-R <= 0 -> [0,-1, 0, 0]  
-P <= 0 -> [0, 0,-1, 0]  
-S <= 0 -> [0, 0, 0,-1]  

The last set are the equality parameters. The ensures that we get a total of 1 when the game is over.  

R + P + S <= 1 ->  [0, 1, 1, 1]  
-R - P - S <= 1 -> [0,-1,-1,-1]  

This results in this matrix:  
[1, 0,-1, 1]  
[1, 1, 0,-1]  
[1,-1, 1, 0]  
[0,-1, 0, 0]  
[0, 0,-1, 0]  
[0, 0, 0,-1]  
[0, 1, 1, 1]  
[0,-1,-1,-1]  

We then transpose it for this library

In [2]:
G = matrix(
    [
        [ 1., 1., 1., 0., 0., 0., 0., 0.],
        [ 0., 1.,-1.,-1., 0., 0., 1.,-1],
        [-1., 0., 1., 0.,-1., 0., 1.,-1.],
        [ 1.,-1., 0., 0., 0.,-1., 1.,-1.]
    ])

**h Matrix**  
The h matrix is the solution to the previous 8 equations. The first 6 equations were <= 0 while the last 2 were the equality equations so you need 1 and -1.

In [4]:
h = matrix([0.,0.,0.,0.,0.,0.,1.,-1.])

**Run the Solver**

In [5]:
sol = solvers.lp(c,G,h, solver=glpksolver)
print(sol['status'])
print(sol['x'])

optimal
[ 2.04e-10]
[ 3.33e-01]
[ 3.33e-01]
[ 3.33e-01]



**Summary**  
If you understood what we were doing you should have expected to get 0 for U and then $\frac{1}{3}$ for throwing each of the 3 types and that is what we got.  

I hope this helps to explain a *slightly* more complex LP than what you would see in an introductory lesson.